In [ ]:
import pandas as pd

In [ ]:
df = pd.read_stata("../data/raw/usa_00026.dta")

df.head()

In [ ]:
dff = df[df['empstat'] == 'unemployed'].groupby(['year','sex','looking'])['perwt'].sum().groupby(['year','sex']).apply(lambda x: x/x.sum()).to_frame()

In [ ]:
%matplotlib inline
dff.xs('no, did not look for work', level = 2).plot.bar()

In [ ]:
df['weightet_inc'] = df['perwt'] * df['inctot']

In [ ]:
df['educd'].unique()

In [ ]:
print(df['educd'].cat.categories)

In [ ]:
df.loc[(df['educd'] > "bachelor's degree"),'educ_attainment'] = 'advanced degree'
df.loc[(df['educd'] == "bachelor's degree"),'educ_attainment'] = 'bachelor'
df.loc[(df['educd'] < "bachelor's degree"),'educ_attainment'] = 'some college'
df.loc[(df['educd'] <= "ged or alternative credential"),'educ_attainment'] = 'hs grad'
df.loc[(df['educd'] < "high school graduate or ged"),'educ_attainment'] = 'no hs'

df['educ_attainment'] = pd.Categorical(df['educ_attainment'], categories=['no hs', 'hs grad', 'some college', 'bachelor', 'advanced degree'], ordered = True)

df.head()

In [ ]:
df['citizen'].cat.categories

In [ ]:
df['nativity'] = 'foreign born'
df.loc[(df['citizen'] < 'naturalized citizen'), 'nativity'] = 'native'

In [ ]:
education = df.groupby(['year','nativity','educ_attainment'])['perwt'].sum().to_frame()
education_pct = education.groupby(['year', 'nativity']).apply(lambda x: x/x.sum())

In [ ]:
education_pct.style.format('{:.2%}')

In [ ]:
import altair as alt

In [ ]:
data = education_pct.reset_index()
data.columns = ['year', 'nativity', 'ed_att', 'pctg']
alt.Chart(data).mark_bar().encode(
    x = alt.X('nativity:O'),
    y = alt.Y('pctg:Q', scale = alt.Scale(domain=[0,1]), axis = alt.Axis(format="%")),
    color = alt.Color('ed_att:N'),
    column = alt.Column('year:O')
)